In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import pandas as pd
import random as rd
import datetime 


from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.stattools import adfuller, acf,pacf, arma_order_select_ic
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs

import warnings 
warnings.filterwarnings('ignore')

In [ ]:
sales = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv")
import warnings
warnings.filterwarnings('ignore')

item_cat = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv')
item = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
sub = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv')
shops = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/shops.csv')
test = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')

In [ ]:
sales.date = sales.date.apply(lambda x:datetime.datetime.strptime(x, '%d.%m.%Y'))

print(sales.info())

In [ ]:
monthly_sales = sales.groupby(["date_block_num", "shop_id", "item_id"])["date", "item_price", "item_cnt_day"].agg({"date":["min", "max"], "item_price":"mean", "item_cnt_day":"sum" })

In [ ]:
monthly_sales.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
x = item.groupby(['item_category_id']).count()
x = x.sort_values(by = 'item_id', ascending = False)
x = x.iloc[0:10].reset_index()

plt.figure(figsize = (8,4))
ax = sns.barplot(x.item_category_id, x.item_id, alpha = 0.8)
plt.title("Items per CAtegory")
plt.ylabel("# of items", fontsize = 12)
plt.xlabel("Category", fontsize = 12)
plt.show()

In [ ]:
ts = sales.groupby(['date_block_num'])["item_cnt_day"].sum()
ts.astype('float')
plt.figure(figsize=(16,8))
plt.title("Total Sales of the Company")
plt.xlabel("Time")
plt.ylabel("Sales")
plt.plot(ts)

In [ ]:
plt.figure(figsize = (16,6))
plt.plot(ts.rolling(window=12, center = False).mean(), label = 'Rolling Mean');
plt.plot(ts.rolling(window=12, center = False).std(), label = 'Rolling sd');
plt.legend()

In [ ]:
def test_stationarity(timeseries):
    
    #Perform Dickey-Fuller test:
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

test_stationarity(ts)

In [ ]:
from pandas import Series as Series

def difference(dataset, interval = 1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
        
        
        
    return  Series(diff)

def inverse_difference(last_ob, value):
    return value + last_ob

In [ ]:
ts=sales.groupby(["date_block_num"])["item_cnt_day"].sum()
ts.astype('float')
plt.figure(figsize=(16,16))
plt.subplot(311)
plt.title('Original')
plt.xlabel('Time')
plt.ylabel('Sales')
plt.plot(ts)
plt.subplot(312)
plt.title('After De-trend')
plt.xlabel('Time')
plt.ylabel('Sales')
new_ts=difference(ts)
plt.plot(new_ts)
plt.plot()

plt.subplot(313)
plt.title('After De-seasonalization')
plt.xlabel('Time')
plt.ylabel('Sales')
new_ts=difference(ts,12)       # assuming the seasonality is 12 months long
plt.plot(new_ts)
plt.plot()

In [ ]:
help(plt.subplot(313))

In [ ]:
print(dataset)

In [ ]:
def tsplot(y, lags = None, figsize = (10,8), style = 'bmh', title = ''):
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
    with plt.style.context(style):
        fig = plt.figure(figsize = figsize)
        
        layout = (3,2)
        ts_ax = plt.subplot2grid(layout, (0,0), colspan = 2)
        acf_ax = plt.subplot2grid(layout, (1,0))
        pacf_ax = plt.subplot2grid(layout, (1,1))
        qq_ax = plt.subplot2grid(layout, (2,0))
        pp_ax = plt.subplot2grid(layout, (2,1))
        
        y.plot(ax = ts_ax)
        ts_ax.set_title(title)
        smt.graphics.plot_acf(y, lags = lags, ax = acf_ax, alpha = 0.5)
        smt.graphics.plot_pacf(y, lags = lags, ax = pacf_ax, alpha = 0.5)
        sm.qqplot(y, line = 's', ax = qq_ax)
        qq_ax = plt.subplot2grid(layout, (2,1))
        
        y.plot(ax = ts_ax)
        ts_ax.set_title(title)
        smt.graphics.plot_acf(y, lags = lags, ax = acf_ax, alpha = 0.5)
        smt.graphics.plot_pacf(y, lags = lags, ax = pacf_ax, alpha = 0.5)
        sm.qqplot(y, line = 's', ax = qq_ax)
        qq_ax.set_title('QQ Plot')
        scs.probplot(y, sparams = (y.mean(), y.std()), plot = pp_ax)
        
        plt.tight_layout()
    return

In [ ]:
np.random.seed(1)
n_samples = int(1000)
a = 0.6
x = w = np.random.normal(size = n_samples)

for t in range(n_samples):
    x[t] = a*x[t-1] + w[t]
    
limit = 12
_ = tsplot(x, lags = limit, title = "AR(1)process")
               

In [ ]:
layout = (3,2)
pacf_ax = plt.subplot2grid(layout, (1, 1))

In [ ]:
 ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)

In [ ]:
help(ts_ax)

In [ ]:
n = int(1000)
alphas = np.array([.444, .333])
betas = np.array([0.])
ar = np.r_[1, -alphas]
ma = np.r_[1, betas]

ar2 = smt.arma_generate_sample(ar = ar, ma = ma, nsample = n)
_ = tsplot(ar2, lags =12, title = "AR(2), process")

In [ ]:
y.plot(ax=ts_ax)
ts_ax.set_title(title)
smt.graphics.plot_acf(y, lags=lags, ax=acf_ax, alpha=0.5)

In [ ]:
n = int(1000)
alphas  =np.array([0.])
betas = np.array([0.6, 0.4])
ar = np.r_[1, -alphas]
ma = np.r_[1, betas]

ma3 = smt.arma_generate_sample(ar = ar, ma = ma, nsample = n)
_ = tsplot(ma3, lags = 12, title = 'MA(2) process')

In [ ]:
k = np.r_[1,2,3]
k

In [ ]:
max_lag = 12
n = int(5000)
burn = int(n/10)
alphas = np.array([0.8, -0.65])
betas = np.array([0.5, -0.7])
ar = np.r_[1, -alphas]
ma = np.r_[1, betas]

arma22 = smt.arma_generate_sample(ar = ar, ma = ma, nsample = n, burnin = burn)
_ = tsplot(arma22, lags = max_lag, title = "ARMA(2, 2) process")